In [5]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np

In [6]:
data = pd.read_csv('../data/raw/df_history_2025-11-03.csv', sep=',')
data.head()

,AuftragsID,BauteilID,Bauteilbezeichnung,Auftragseingang,Priorität,Auftragsende_SOLL,Arbeitsschritt,Arbeitsschrittbezeichnung,AFO_Start_SOLL,AFO_Ende_SOLL,AFO_Start_IST,AFO_Ende_IST,MaschinenID,Maschinenbezeichnung,Kapazität,Auftragsende_IST,AFO_Dauer_IST_Stunde,Auftragsdauer_IST_Tag
0,1,1,Steuerventilmodul,2013-10-29,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN,2014-05-07 08:39:00,0.02,136.21
1,2,1,Steuerventilmodul,2013-08-16,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN,2014-05-07 09:04:00,0.02,188.26
2,3,1,Steuerventilmodul,2013-08-05,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN,2014-05-07 11:25:00,0.02,197.55
3,4,1,Steuerventilmodul,2013-10-12,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN,2014-05-07 09:19:00,0.02,147.29
4,5,1,Steuerventilmodul,2013-10-03,1,2014-01-01 11:32:00,1,Info,2014-01-01 07:00:00,2014-01-01 07:01:00,2014-01-01 07:00:00,2014-01-01 07:01:00,NaN,NaN,NaN,2014-05-07 09:07:00,0.02,154.26


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465945 entries, 0 to 1465944
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   AuftragsID                 1465945 non-null  int64  
 1   BauteilID                  1465945 non-null  int64  
 2   Bauteilbezeichnung         1465945 non-null  object 
 3   Auftragseingang            1465945 non-null  object 
 4   Priorität                  1465945 non-null  int64  
 5   Auftragsende_SOLL          1465945 non-null  object 
 6   Arbeitsschritt             1465945 non-null  int64  
 7   Arbeitsschrittbezeichnung  1465945 non-null  object 
 8   AFO_Start_SOLL             1465945 non-null  object 
 9   AFO_Ende_SOLL              1465945 non-null  object 
 10  AFO_Start_IST              1465945 non-null  object 
 11  AFO_Ende_IST               1465945 non-null  object 
 12  MaschinenID                1178102 non-null  object 
 13  Maschinenbez

In [8]:
data["AuftragsID"].nunique(), data["BauteilID"].nunique()

# Also im Schnitt ca. 10 Arbeitsschritte pro Auftrag. 

(150368, 3)

In [9]:
# Zeitspalten von Text (object) in echtes Datumsformat (datetime64) umwandeln,
# damit Zeitdifferenzen, Vergleiche und Gruppierungen möglich sind.
# errors="coerce" wandelt fehlerhafte Werte in NaT (Not a Time) statt Fehler auszulösen.
time_cols = ["Auftragseingang", "Auftragsende_SOLL",
              "AFO_Start_SOLL", "AFO_Ende_SOLL",
              "AFO_Start_IST", "AFO_Ende_IST", "Auftragsende_IST" ]

for col in time_cols:
    data[col] = pd.to_datetime(data[col], errors="coerce")

data[time_cols].isna().sum()

Auftragseingang          0
Auftragsende_SOLL        0
AFO_Start_SOLL           0
AFO_Ende_SOLL            0
AFO_Start_IST            0
AFO_Ende_IST             0
Auftragsende_IST     58851
dtype: int64

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465945 entries, 0 to 1465944
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   AuftragsID                 1465945 non-null  int64         
 1   BauteilID                  1465945 non-null  int64         
 2   Bauteilbezeichnung         1465945 non-null  object        
 3   Auftragseingang            1465945 non-null  datetime64[ns]
 4   Priorität                  1465945 non-null  int64         
 5   Auftragsende_SOLL          1465945 non-null  datetime64[ns]
 6   Arbeitsschritt             1465945 non-null  int64         
 7   Arbeitsschrittbezeichnung  1465945 non-null  object        
 8   AFO_Start_SOLL             1465945 non-null  datetime64[ns]
 9   AFO_Ende_SOLL              1465945 non-null  datetime64[ns]
 10  AFO_Start_IST              1465945 non-null  datetime64[ns]
 11  AFO_Ende_IST               1465945 no

In [11]:
raw = pd.read_csv('../data/raw/df_history_2025-11-03.csv', sep=',')

# Zeilen finden, die nach der Umwandlung NaT wurden
mask_invalid = data["AFO_Start_IST"].isna()

# Vergleich der Originalwerte in diesen Zeilen
raw.loc[mask_invalid, "AFO_Start_IST"].unique()[:20]



array([], dtype=object)

In [12]:
raw.loc[mask_invalid].head(20)

,AuftragsID,BauteilID,Bauteilbezeichnung,Auftragseingang,Priorität,Auftragsende_SOLL,Arbeitsschritt,Arbeitsschrittbezeichnung,AFO_Start_SOLL,AFO_Ende_SOLL,AFO_Start_IST,AFO_Ende_IST,MaschinenID,Maschinenbezeichnung,Kapazität,Auftragsende_IST,AFO_Dauer_IST_Stunde,Auftragsdauer_IST_Tag


In [13]:
raw.loc[mask_invalid, "AFO_Start_IST"].value_counts().head(10)

Series([], Name: count, dtype: int64)

In [14]:
raw.loc[mask_invalid & (raw["AFO_Start_IST"].astype(str).str.strip() == "2000-01-01")]


,AuftragsID,BauteilID,Bauteilbezeichnung,Auftragseingang,Priorität,Auftragsende_SOLL,Arbeitsschritt,Arbeitsschrittbezeichnung,AFO_Start_SOLL,AFO_Ende_SOLL,AFO_Start_IST,AFO_Ende_IST,MaschinenID,Maschinenbezeichnung,Kapazität,Auftragsende_IST,AFO_Dauer_IST_Stunde,Auftragsdauer_IST_Tag


In [15]:
raw = pd.read_csv('../data/raw/df_history_2025-11-03.csv', sep=',')

# Zeilen finden, die nach der Umwandlung NaT wurden
mask_invalid = data["AFO_Ende_IST"].isna()

# Vergleich der Originalwerte in diesen Zeilen
raw.loc[mask_invalid, "AFO_Ende_IST"].unique()[:20]


array([], dtype=object)

In [16]:
raw.loc[mask_invalid, "AFO_Ende_IST"].value_counts().head(10)

Series([], Name: count, dtype: int64)

In [17]:
# Zeilen mit fehlenden (NaT) IST-Zeitstempeln entfernen
data_clean = data.dropna(subset=["AFO_Start_IST", "AFO_Ende_IST"]).copy()

# Kontrolle: Wie viele Zeilen wurden entfernt?
removed_rows = len(data) - len(data_clean)
print(f"Entfernte Zeilen: {removed_rows}")
print(f"Verbleibende Zeilen: {len(data_clean)}")

Entfernte Zeilen: 0
Verbleibende Zeilen: 1465945


In [18]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465945 entries, 0 to 1465944
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   AuftragsID                 1465945 non-null  int64         
 1   BauteilID                  1465945 non-null  int64         
 2   Bauteilbezeichnung         1465945 non-null  object        
 3   Auftragseingang            1465945 non-null  datetime64[ns]
 4   Priorität                  1465945 non-null  int64         
 5   Auftragsende_SOLL          1465945 non-null  datetime64[ns]
 6   Arbeitsschritt             1465945 non-null  int64         
 7   Arbeitsschrittbezeichnung  1465945 non-null  object        
 8   AFO_Start_SOLL             1465945 non-null  datetime64[ns]
 9   AFO_Ende_SOLL              1465945 non-null  datetime64[ns]
 10  AFO_Start_IST              1465945 non-null  datetime64[ns]
 11  AFO_Ende_IST               1465945 no

In [19]:
data_clean["AFO_Start_IST"].min(), data_clean["AFO_Ende_IST"].max()


(Timestamp('2000-01-01 00:00:00'), Timestamp('2024-03-01 14:30:00'))

In [20]:
placeholder_counts = pd.Series({
    "AFO_Start_IST == 2000-01-01": (data["AFO_Start_IST"] == pd.Timestamp("2000-01-01")).sum(),
    "AFO_Start_IST == 2100-01-01": (data["AFO_Start_IST"] == pd.Timestamp("2100-01-01")).sum(),
    "AFO_Ende_IST == 2000-01-01": (data["AFO_Ende_IST"] == pd.Timestamp("2000-01-01")).sum(),
    "AFO_Ende_IST == 2100-01-01": (data["AFO_Ende_IST"] == pd.Timestamp("2100-01-01")).sum()
})

print(placeholder_counts)

AFO_Start_IST == 2000-01-01    281
AFO_Start_IST == 2100-01-01      0
AFO_Ende_IST == 2000-01-01       0
AFO_Ende_IST == 2100-01-01       0
dtype: int64


In [21]:
mask_future = (
    (data["AFO_Start_IST"] == pd.Timestamp("2100-01-01")) |
    (data["AFO_Start_IST"] == pd.Timestamp("2000-01-01")) |
    (data["AFO_Ende_IST"] == pd.Timestamp("2100-01-01"))
)

data_no_future = data_clean.loc[~mask_future].copy()

print(f"Entfernte Zeilen mit 2100-01-01: {mask_future.sum()}")
print(f"Verbleibende Zeilen: {len(data_no_future)}")
print((data_no_future["AFO_Start_IST"] == pd.Timestamp("2100-01-01")).sum())
print((data_no_future["AFO_Start_IST"] == pd.Timestamp("2000-01-01")).sum())
print((data_no_future["AFO_Ende_IST"] == pd.Timestamp("2100-01-01")).sum())

Entfernte Zeilen mit 2100-01-01: 281
Verbleibende Zeilen: 1465664
0
0
0


In [22]:
(data_no_future["AFO_Ende_IST"] < data_no_future["AFO_Start_IST"]).sum()

np.int64(0)

In [23]:
(data_no_future["AFO_Ende_SOLL"] < data_no_future["AFO_Start_SOLL"]).sum()

np.int64(0)

In [24]:
data_no_future["AFO_Start_IST"].min(), data_no_future["AFO_Ende_IST"].max()

(Timestamp('2014-01-01 07:00:00'), Timestamp('2024-03-01 14:30:00'))

In [25]:
data_no_future = data_no_future.reset_index(drop=True)

In [26]:
data_no_future.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465664 entries, 0 to 1465663
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   AuftragsID                 1465664 non-null  int64         
 1   BauteilID                  1465664 non-null  int64         
 2   Bauteilbezeichnung         1465664 non-null  object        
 3   Auftragseingang            1465664 non-null  datetime64[ns]
 4   Priorität                  1465664 non-null  int64         
 5   Auftragsende_SOLL          1465664 non-null  datetime64[ns]
 6   Arbeitsschritt             1465664 non-null  int64         
 7   Arbeitsschrittbezeichnung  1465664 non-null  object        
 8   AFO_Start_SOLL             1465664 non-null  datetime64[ns]
 9   AFO_Ende_SOLL              1465664 non-null  datetime64[ns]
 10  AFO_Start_IST              1465664 non-null  datetime64[ns]
 11  AFO_Ende_IST               1465664 no

In [27]:

data_no_future["Maschinenbezeichnung"].value_counts().head(10)


Maschinenbezeichnung
Fronius        148537
EWM            140628
Deckel Maho    135182
DMG Mori        82316
Lorch            9591
XXX               738
Name: count, dtype: int64

In [28]:
data_no_future["Bauteilbezeichnung"].value_counts(normalize=True)


Bauteilbezeichnung
Schwenkzylinder      0.562885
Steuerventilmodul    0.404043
Daempfungseinheit    0.033071
Name: proportion, dtype: float64

In [29]:
data_no_future["Arbeitsschrittbezeichnung"].value_counts().head(15)

Arbeitsschrittbezeichnung
Transport                 660973
Info                      146598
End                       141822
Richten                    75702
Mechanisch Bearbeiten      75536
Aufspannen                 75471
Abspannen-Nacharbeiten     73354
Schweißen                  64226
Heften-Schweißen           64168
Nachschweißen              61845
manuelle Nacharbeit        12741
Verputzen-Aufspannen        6614
Lackieren                   6614
Name: count, dtype: int64

In [30]:
data_no_future.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465664 entries, 0 to 1465663
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   AuftragsID                 1465664 non-null  int64         
 1   BauteilID                  1465664 non-null  int64         
 2   Bauteilbezeichnung         1465664 non-null  object        
 3   Auftragseingang            1465664 non-null  datetime64[ns]
 4   Priorität                  1465664 non-null  int64         
 5   Auftragsende_SOLL          1465664 non-null  datetime64[ns]
 6   Arbeitsschritt             1465664 non-null  int64         
 7   Arbeitsschrittbezeichnung  1465664 non-null  object        
 8   AFO_Start_SOLL             1465664 non-null  datetime64[ns]
 9   AFO_Ende_SOLL              1465664 non-null  datetime64[ns]
 10  AFO_Start_IST              1465664 non-null  datetime64[ns]
 11  AFO_Ende_IST               1465664 no

In [31]:
data_no_future = data_no_future[data_no_future["Auftragseingang"] >= pd.Timestamp("2010-01-01")]

In [32]:
placeholder_counts = pd.Series({
    "AFO_Start_IST == 2000-01-01": (data_no_future["AFO_Start_IST"] == pd.Timestamp("2000-01-01")).sum(),
    "AFO_Start_IST == 2100-01-01": (data_no_future["AFO_Start_IST"] == pd.Timestamp("2100-01-01")).sum(),
    "AFO_Ende_IST == 2000-01-01": (data_no_future["AFO_Ende_IST"] == pd.Timestamp("2000-01-01")).sum(),
    "AFO_Ende_IST == 2100-01-01": (data_no_future["AFO_Ende_IST"] == pd.Timestamp("2100-01-01")).sum(),
    "Auftragseingang < 2010-01-01": (data_no_future["Auftragseingang"] < pd.Timestamp("2010-01-01")).sum(),
    "Auftragsende_SOLL > 2026-01-01": (data_no_future["Auftragseingang"] > pd.Timestamp("2026-01-01")).sum()
     


})

print(placeholder_counts)

AFO_Start_IST == 2000-01-01       0
AFO_Start_IST == 2100-01-01       0
AFO_Ende_IST == 2000-01-01        0
AFO_Ende_IST == 2100-01-01        0
Auftragseingang < 2010-01-01      0
Auftragsende_SOLL > 2026-01-01    0
dtype: int64


In [33]:
data_no_future.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465664 entries, 0 to 1465663
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   AuftragsID                 1465664 non-null  int64         
 1   BauteilID                  1465664 non-null  int64         
 2   Bauteilbezeichnung         1465664 non-null  object        
 3   Auftragseingang            1465664 non-null  datetime64[ns]
 4   Priorität                  1465664 non-null  int64         
 5   Auftragsende_SOLL          1465664 non-null  datetime64[ns]
 6   Arbeitsschritt             1465664 non-null  int64         
 7   Arbeitsschrittbezeichnung  1465664 non-null  object        
 8   AFO_Start_SOLL             1465664 non-null  datetime64[ns]
 9   AFO_Ende_SOLL              1465664 non-null  datetime64[ns]
 10  AFO_Start_IST              1465664 non-null  datetime64[ns]
 11  AFO_Ende_IST               1465664 no

In [34]:
data_no_future.to_csv('../data/processed/data_cleaned_timestamp_1.csv', index=False)

# Summary of the Data Cleaning Steps

- **Loaded the raw dataset** and inspected structure, data types, and uniqueness of key identifiers.

- **Converted timestamp columns** (e.g., `Auftragseingang`, `AFO_Start_IST`, etc.) from text into proper `datetime64` objects  
  → using `errors="coerce"` so invalid values became `NaT` instead of raising errors.

- **Identified rows with invalid timestamps** by comparing parsed values with the original raw data.  
  → Detected placeholder dates such as `2000-01-01` and `2100-01-01`.

- **Removed rows with missing IST timestamps**, i.e.  
  rows where `AFO_Start_IST` or `AFO_Ende_IST` converted to `NaT`.

- **Filtered out rows containing placeholder or unrealistic future dates**, specifically:
  - `2000-01-01`
  - `2100-01-01`

- **Checked logical consistency**, ensuring:
  - End timestamps are never earlier than start timestamps (IST and SOLL).
  - No unrealistic or future dates remain in the dataset.

- **Filtered out all orders before 2010** to keep only relevant, modern historical data.

- **Reset the index** and inspected the distributions of machines, parts, and process steps.

- **Saved the cleaned dataset** as  
  `data_cleaned_timestamp_1.csv`.
